<a href="https://colab.research.google.com/github/lotrmay/7SWI1_project/blob/main/bc_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

In [ ]:
!pip install wandb
!wandb login
import wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
%tensorflow_version 2.x
import tensorflow as tf
import pickle
import keras
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn
from sklearn.preprocessing import StandardScaler


CSV_COLUMN_NAMES=['Odds_firstTeam','Odds_secondTeam','Rank_firstTeam','Rank_secondTeam','WinRate_firstTeam','WinRate_secondTeam','PistolWinRate_firstTeam','PistolWinRate_secondTeam',
                  'playerAARating','playerAADpr','playerAAKast','playerAAImpact','playerAAAdr','playerAAKpr','playerAAHs','playerAAKD','playerAAGrenadeDmg',
                  'playerABRating','playerABDpr','playerABKast','playerABImpact','playerABAdr','playerABKpr','playerABHs','playerABKD','playerABGrenadeDmg',
                  'playerACRating','playerACDpr','playerACKast','playerACImpact','playerACAdr','playerACKpr','playerACHs','playerACKD','playerACGrenadeDmg',
                  'playerADRating','playerADDpr','playerADKast','playerADImpact','playerADAdr','playerADKpr','playerADHs','playerADKD','playerADGrenadeDmg',
                  'playerAERating','playerAEDpr','playerAEKast','playerAEImpact','playerAEAdr','playerAEKpr','playerAEHs','playerAEKD','playerAEGrenadeDmg',
                  'playerBARating','playerBADpr','playerBAKast','playerBAImpact','playerBAAdr','playerBAKpr','playerBAHs','playerBAKD','playerBAGrenadeDmg',
                  'playerBBRating','playerBBDpr','playerBBKast','playerBBImpact','playerBBAdr','playerBBKpr','playerBBHs','playerBBKD','playerBBGrenadeDmg',
                  'playerBCRating','playerBCDpr','playerBCKast','playerBCImpact','playerBCAdr','playerBCKpr','playerBCHs','playerBCKD','playerBCGrenadeDmg',
                  'playerBDRating','playerBDDpr','playerBDKast','playerBDImpact','playerBDAdr','playerBDKpr','playerBDHs','playerBDKD','playerBDGrenadeDmg',
                  'playerBERating','playerBEDpr','playerBEKast','playerBEImpact','playerBEAdr','playerBEKpr','playerBEHs','playerBEKD','playerBEGrenadeDmg','matchFormat','Match_link','Result','team_one_name','team_two_name']

#datasetOne=pd.read_csv('/content/drive/MyDrive/data/betterData.csv',sep=";",names=CSV_COLUMN_NAMES,error_bad_lines=False,header=None)#vytvoří dataframe z našeho csv souboru
#datasetTwo=pd.read_csv('/content/drive/MyDrive/data/dataWithPredictedOdds.csv',sep=";",names=CSV_COLUMN_NAMES,error_bad_lines=False,header=0)
#dataset=pd.concat([datasetOne,datasetTwo])
dataset=pd.read_csv('/content/drive/MyDrive/data/betterData.csv',sep=";",names=CSV_COLUMN_NAMES,error_bad_lines=False,header=None)

print(dataset.shape)#vypíše nám dimenzionalitu našeho dataframu např.(2, 3) 2 řádky 3 sloupce
#print(dataset.describe())
#dataBO1
#dataBO3
#betterData
#easyData
#dataWithPredictedOdds

#následující 2 řádky nám upraví dva sloupce z textových na číselné formáty (category datatype)
dataset['team_one_name']=pd.Categorical(dataset['team_one_name']).codes #sníží využití paměti z 1.2MB na 0.03 MB viz: https://towardsdatascience.com/staying-sane-while-adopting-pandas-categorical-datatypes-78dbd19dcd8a
dataset['team_two_name']=pd.Categorical(dataset['team_two_name']).codes
dataset['matchFormat']=pd.Categorical(dataset['matchFormat']).codes


#Odstraním z dataframu následující sloupce (odkaz na zápas a jména týmů), jelikož jsem je využíval pouze při sběru dat
dataset.pop('Match_link')
dataset.pop('team_one_name')
dataset.pop('team_two_name')
dataset.pop('matchFormat')

#https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
#frac=1 znamená, vrať všechny řádky
dataset = dataset.sample(frac=1).reset_index(drop=True) #náhodně zamícháme řádky (můžeme využít seed)

main_scaler=sklearn.preprocessing.StandardScaler()#standardscaler narozdíl od minmaxscaleru vykazuje o 2 procenta lepší přesnost, navíc je logické ho použít
y = dataset['Result'].values
dataset.pop('Result')
X = dataset.values

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


(18859, 103)


In [21]:
from sklearn.ensemble import IsolationForest 
from sklearn.svm import OneClassSVM 
from sklearn.covariance import EllipticEnvelope 
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(contamination=0.05)
yhat = lof.fit_predict(X)

mask = yhat != -1
X, y = X[mask, :], y[mask]
print(X.shape, y.shape)

(17916, 98) (17916,)


In [22]:
#rozdělíme náš dataframe na trénovací, testovací a validační dataset
#testovací dataset bude 10%
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20)
#validační set bude 10% 
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X_test, y_test, test_size=0.5)

print(X_train.shape) 
print(X_test.shape)
print(X_val.shape)

main_scaler.fit(X_train)

X_train=main_scaler.transform(X_train)#:https://stackoverflow.com/questions/49444262/normalize-data-before-or-after-split-of-training-and-testing-data
X_test=main_scaler.transform(X_test)
X_val=main_scaler.transform(X_val)

(17916, 98)
(17916,)
(14332, 98)
(1792, 98)
(1792, 98)


In [ ]:
!pip install tensorflow-text

In [23]:
import random
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataset, labels, batch_size=32):
        self.dataset = dataset 
        self.batch_size = batch_size
        self.labels = labels
        self.indexes = self.generateIDs(dataset.shape[0])
        self.mean=self.dataset.mean(axis=0)
        self.std=self.dataset.std(axis=0)
        self.on_epoch_end()

    def generateIDs(self,length):
      ret=list()
      for i in range(0,length):
        ret.append(i)
      return ret

    def __len__(self):
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
      np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
      mu=np.mean(self.dataset, axis = 0)
      sigma=np.std(self.dataset, axis=0)
      Xret = np.empty(shape=(self.batch_size,98))
      yret = np.empty((self.batch_size), dtype=int)
      for i, ID in enumerate(list_IDs_temp):
        Xret[i] = self.dataset[ID]
        yret[i] = self.labels[ID]
        if(random.random()<=0.3):#po tomhle udělat i ranodm na augmentaci, kde by mohly být dvě pole jedno 1-5 další 5-10 dát random shuffle a v tom pořadí tam přidat hráče a pak dát ještě nějakou pravděpodobnost na flipnutí?
          augmentedAttr=np.random.randint(3,98,size=random.randint(0, 35))#augmentovat odds? míra augmentace?
          for attr in augmentedAttr:
            gaussValue=random.gauss(self.mean[attr], self.std[attr])
            Xret[i][attr]=gaussValue
        if(random.random()<=0.5):
          flippedXret=Xret[i][[1,0,3,2,5,4,7,6]]
          flippedXret=np.concatenate((flippedXret,Xret[i][53:98]))
          flippedXret=np.concatenate((flippedXret,Xret[i][8:53]))
          Xret[i]=flippedXret
          yret[i]=abs(yret[i]-1)
      Xret=returnTextForm(Xret)
      return Xret, yret

In [24]:
import tensorflow_hub as hub
import tensorflow_text as text

def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer( "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2",trainable=True)
  outputs = encoder(encoder_inputs)
  l = outputs['pooled_output']
  l = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(l)
  return tf.keras.Model(text_input, l)


In [25]:
y_train = np.array(y_train)
y_test=np.array(y_test)
y_val=np.array(y_val)

def returnTextForm(input):
    ret = []
    for row in input:
        combined = ""
        combined += '{} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} '.format(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14],row[15],row[16],row[17],row[18],row[19],row[20],row[21],row[22],row[23],row[24],row[25],row[26],row[27],row[28],row[29],row[30],row[31],row[32],row[33],row[34],row[35],row[36],row[37],row[38],row[39],row[40],row[41],row[42],row[43],row[44],row[45],row[46],row[47],row[48],row[49],row[50],row[51],row[52],row[53],row[54],row[55],row[56],row[57],row[58],row[59],row[60],row[61],row[62],row[63],row[64],row[65],row[66],row[67],row[68],row[69],row[70],row[71],row[72],row[73],row[74],row[75],row[76],row[77],row[78],row[79],row[80],row[81],row[82],row[83],row[84],row[85],row[86],row[87],row[88],row[89],row[90],row[91],row[92],row[93],row[94],row[95],row[96],row[97])
        ret.append(combined)
    return np.array(ret)

X_test=returnTextForm(X_test)
X_val=returnTextForm(X_val)

In [ ]:
import optuna
def objective(trial):
    bs  = trial.suggest_categorical('batch_size', [32,64,128,256,512])
    opt_name = trial.suggest_categorical('optimizer', ['Adadelta','Adagrad'])#'Adam','Adagrad
    lr=trial.suggest_categorical('learning_rate', [1.0])
    m_mul_value=trial.suggest_uniform('m_mul', 0.6, 0.95)
    min_learning_rate=0.01#round(trial.suggest_uniform('minimum_learning_rate', 1e-6, lr),3)
    opt=keras.optimizers.get(opt_name)
    opt.learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(lr, round((len(X_train)/bs*50)/15), t_mul=2.0, m_mul=m_mul_value, alpha=min_learning_rate)

    classifier_model = build_classifier_model()
    dataGen=DataGenerator(X_train,y_train,bs)
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,verbose=0)#patience-kolik epoch se nezmění val_loss pak stop
   
    classifier_model.compile(optimizer=opt,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), #https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
                  metrics=['binary_accuracy'])
    
    classifier_model.fit(dataGen,epochs=50,validation_data=(X_val,y_val),callbacks=[early_stopping_callback])
    return classifier_model.evaluate(X_test, y_test, verbose=0)[1]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

print('params:', study.best_trial.params)
print('best accuracy:', study.best_value)


In [ ]:
class LRLogger(tf.keras.callbacks.Callback):
    def __init__(self, optimizer):
      super(LRLogger, self).__init__()
      self.optimizer = optimizer

    def on_epoch_begin(self, epoch, logs):
      lr = self.optimizer.learning_rate(self.optimizer.iterations)
      wandb.log({"lr": lr}, commit=False)

hyperparameter_defaults = dict (
  optimizer="Adadelta",
  learning_rate = 1.0,
  min_learning_rate=0.001,
  batch_size = 128,
  epochs = 100,
  reduce_factor=0.95
)#počet stepů na epochu * počet chtěných epoch / 15

wandb.init(config=hyperparameter_defaults, entity="lotrmay", project="bc")
classifier_model = build_classifier_model()
dataGen=DataGenerator(X_train,y_train,hyperparameter_defaults.get("batch_size"))
opt=keras.optimizers.get(hyperparameter_defaults.get("optimizer"))
lr=tf.keras.optimizers.schedules.CosineDecayRestarts(hyperparameter_defaults.get("learning_rate"), round((len(X_train)/hyperparameter_defaults.get("batch_size")*hyperparameter_defaults.get("epochs"))/15), t_mul=2.0, m_mul=hyperparameter_defaults.get("reduce_factor"), alpha=hyperparameter_defaults.get("min_learning_rate"))
opt.learning_rate=lr
classifier_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['binary_accuracy'])

pickle.dump(main_scaler, open("/content/drive/MyDrive/data/recent_scaler.pkl", 'wb'))
logging_callback = wandb.keras.WandbCallback()
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/data/recent_model.h5", monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15,verbose=1)

dataGen=DataGenerator(X_train,y_train,hyperparameter_defaults.get("batch_size"))
history = classifier_model.fit(dataGen, epochs=hyperparameter_defaults.get("epochs"),validation_data=(X_val, y_val), callbacks=[checkpoint_callback,early_stopping_callback,LRLogger(opt),logging_callback])#steps
model= tf.keras.models.load_model(("/content/drive/MyDrive/data/recent_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer})
scaler=pickle.load(open("/content/drive/MyDrive/data/recent_scaler.pkl", 'rb'))

scaledData=scaler.transform(X)
scaledData=returnTextForm(scaledData)
pred_test= model.predict(scaledData)
scores2 = model.evaluate(scaledData, y, verbose=1)
print('Accuracy on the whole dataset: {:.2%} \n Error on the whole dataset: {:.2%}'.format(scores2[1], 1 - scores2[1]))

plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('binary_accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
wandb.finish()


lr,▁
lr,1.0


Epoch 1/100
111/111 [==============================] - ETA: 0s - loss: 0.6703 - binary_accuracy: 0.6058
Epoch 1: val_loss improved from inf to 0.63598, saving model to /content/drive/MyDrive/data/recent_model.h5
111/111 [==============================] - 239s 2s/step - loss: 0.6703 - binary_accuracy: 0.6058 - val_loss: 0.6360 - val_binary_accuracy: 0.6769 - _timestamp: 1650437531.0000 - _runtime: 249.0000
Epoch 2/100
  5/111 [>.............................] - ETA: 3:30 - loss: 0.6475 - binary_accuracy: 0.6219

Testování embedded modelů:
























In [ ]:
models=list()
models.append(keras.models.load_model("/content/drive/MyDrive/data/new/m1.h5"))
models.append(keras.models.load_model("/content/drive/MyDrive/data/new/m2.h5"))
models.append(keras.models.load_model("/content/drive/MyDrive/data/new/m3.h5"))

scalers=list()

scalers.append(pickle.load(open("/content/drive/MyDrive/data/new/s1.pkl", 'rb')))
scalers.append(pickle.load(open("/content/drive/MyDrive/data/new/s2.pkl", 'rb')))
scalers.append(pickle.load(open("/content/drive/MyDrive/data/new/s3.pkl", 'rb')))


data=scalers[0].transform(X)
predictions=models[0].predict(data)

data=scalers[1].transform(X)
predictions2=models[1].predict(data)

data=scalers[2].transform(X)
predictions3=models[2].predict(data)


predictionList=list()
for i in range(y.shape[0]):
  yhats=list()
  yhats.append(predictions[i])
  yhats.append(predictions2[i])
  yhats.append(predictions3[i])


  yhats = np.array(yhats)
  outcomes = np.mean(yhats)
  predictionList.append(round(outcomes))

count = np.count_nonzero(predictionList == y)
print(count/y.shape[0])
print('Accuracy on the whole dataset: {:.2%} \n Error on the whole dataset: {:.2%}'.format(count/y.shape[0], 1 - count/y.shape[0]))

In [ ]:
models=list()
models.append(tf.keras.models.load_model(("/content/drive/MyDrive/data/best_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer}))
models.append(tf.keras.models.load_model(("/content/drive/MyDrive/data/test/recent_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer}))
models.append(tf.keras.models.load_model(("/content/drive/MyDrive/data/test/recent_model2.h5"),custom_objects={'KerasLayer':hub.KerasLayer}))

scalers=list()
scalers.append(pickle.load(open("/content/drive/MyDrive/data/best_scaler.pkl", 'rb')))
scalers.append(pickle.load(open("/content/drive/MyDrive/data/test/recent_scaler.pkl", 'rb')))
scalers.append(pickle.load(open("/content/drive/MyDrive/data/test/recent_scaler2.pkl", 'rb')))


data=scalers[0].transform(X)
data=returnTextForm(data)
predictions=models[0].predict(data)

data=scalers[1].transform(X)
data=returnTextForm(data)
predictions2=models[1].predict(data)

data=scalers[2].transform(X)
data=returnTextForm(data)
predictions3=models[2].predict(data)


In [ ]:

predictionList=list()
for i in range(y.shape[0]):
  yhats=list()
  yhats.append(predictions[i])
  #yhats.append(predictions2[i])
  yhats.append(predictions3[i])

  yhats = np.array(yhats)
  outcomes = np.mean(yhats)
  predictionList.append(round(outcomes))

count = np.count_nonzero(predictionList == y)
print(count/y.shape[0])
print('Accuracy on the whole dataset: {:.2%} \n Error on the whole dataset: {:.2%}'.format(count/y.shape[0], 1 - count/y.shape[0]))


0.6917838803304309
Accuracy on the whole dataset: 69.18% 
 Error on the whole dataset: 30.82%


In [ ]:
Xnew=np.array([[2.04,1.81,29,36,0.38,0.42,0.484,0.428,1.08,0.68,0.72,1.01,78.3,0.73,0.39,1.07,4.4,1.03,0.66,0.68,0.99,74.3,0.68,0.56,1.03,3.9,1.01,0.71,0.7,0.95,76.9,0.65,0.5,0.91,2.6,0.99,0.67,0.66,1.01,68.0,0.67,0.36,1.01,3.4,0.91,0.7,0.69,0.81,66.6,0.56,0.5,0.80,5.2,1.04,0.67,0.69,1.04,71.1,0.69,0.26,1.03,6.4,1.04,0.68,0.67,1.07,75.1,0.68,0.56,1.00,7.4,1.04,0.72,0.69,1.05,75.9,0.71,0.64,0.99,4.4,0.88,0.71,0.65,0.78,67.9,0.58,0.52,0.82,4.3,0.86,0.72,0.65,0.86,64.5,0.56,0.48,0.78,6.6],
               [1.22,4,1,19,0.77,0.62,0.606,0.591,1.18,0.57,0.73,1.17,75.2,0.75,0.46,1.33,1.9,1.16,0.56,0.77,1.01,71.9,0.71,0.28,1.28,3.9,1.13,0.56,0.75,0.98,75.8,0.66,0.59,1.19,5.3,1.11,0.67,0.72,1.11,79.6,0.7,0.47,1.04,6.0,0.93,0.68,0.66,0.94,67.9,0.57,0.48,0.85,6.6,1.22,0.56,0.74,1.28,75.6,0.74,0.23,1.33,3.6,1.2,0.72,0.71,1.32,90.3,0.77,0.55,1.08,9.5,1.09,0.63,0.72,1.04,74.4,0.7,0.52,1.12,3.0,1.02,0.63,0.7,0.93,70.0,0.62,0.47,1.00,7.2,0.94,0.67,0.69,0.78,66.5,0.62,0.63,0.92,4.0],
               [3.18,1.33,107,51,0.58,0.48,0.5,0.472,1.26,0.62,0.75,1.34,83.0,0.81,0.27,1.30,4.9,1.1,0.72,0.7,1.18,84.6,0.73,0.37,1.00,9.8,1.06,0.72,0.7,1.13,77.3,0.73,0.4,1.02,4.5,1.02,0.67,0.69,0.98,74.6,0.67,0.46,1.00,6.4,0.89,0.63,0.69,0.69,60.9,0.55,0.56,0.87,4.0,1.12,0.61,0.71,1.1,73.9,0.72,0.28,1.18,3.6,1.07,0.7,0.68,1.1,79.0,0.71,0.48,1.02,5.3,1.07,0.7,0.66,1.13,79.9,0.72,0.54,1.03,2.8,0.98,0.68,0.68,0.93,70.2,0.65,0.53,0.95,3.7,1.06,0.67,0.68,1.06,75.75,0.7,0.46,1.05,3.85],
               [1.4,2.75,54,100,0.64,0.46,0.575,0.5,1.13,0.63,0.71,1.16,76.4,0.7,0.46,1.11,5.2,1.09,0.63,0.7,1.05,75.5,0.69,0.49,1.10,5.3,1.02,0.67,0.68,1.06,73.8,0.62,0.55,0.93,7.3,1.08,0.64,0.7,1.09,75.23,0.67,0.5,1.05,5.93,1.08,0.64,0.7,1.09,75.23,0.67,0.5,1.05,5.93,1.2,0.68,0.71,1.26,86.6,0.83,0.45,1.23,4.0,1.03,0.65,0.72,0.97,70.8,0.64,0.49,0.98,4.7,1.02,0.66,0.7,0.99,70.8,0.66,0.4,1.01,5.6,0.93,0.74,0.66,0.98,72.1,0.61,0.4,0.83,6.8,1.04,0.68,0.7,1.05,75.07,0.68,0.43,1.01,5.27],
               [1.71,2.08,121,140,0.8,0.79,0.5,0.5,1.48,0.61,0.75,1.72,99.8,0.92,0.27,1.51,3.1,1.22,0.6,0.79,1.12,78.9,0.78,0.52,1.31,4.0,1.21,0.58,0.78,1.02,82.3,0.73,0.57,1.27,5.6,1.3,0.6,0.77,1.29,87.0,0.81,0.45,1.36,4.23,1.3,0.6,0.77,1.29,87.0,0.81,0.45,1.36,4.23,1.27,0.58,0.74,1.31,86.4,0.79,0.41,1.35,5.7,1.2,0.58,0.77,1.12,77.6,0.75,0.26,1.29,6.1,1.11,0.68,0.72,1.19,79.2,0.71,0.4,1.04,9.8,1.1,0.6,0.71,1.06,74.0,0.69,0.43,1.14,8.3,1.05,0.64,0.73,0.93,74.3,0.69,0.47,1.08,6.5]])
scaler=pickle.load(open("/content/drive/MyDrive/data/best_scaler.pkl", 'rb'))
model=tf.keras.models.load_model(("/content/drive/MyDrive/data/best_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer})
Xnew=scaler.transform(Xnew)
Xnew=returnTextForm(Xnew)
model.predict(Xnew)

array([[0.45843104],
       [0.9150423 ],
       [0.34710377],
       [0.37569317],
       [0.53999895],
       [0.25977474],
       [0.52886564],
       [0.4493159 ],
       [0.11985713],
       [0.4891027 ],
       [0.518847  ],
       [0.3475605 ],
       [0.36075062]], dtype=float32)

In [ ]:
model= tf.keras.models.load_model(("/content/drive/MyDrive/data/recent_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer})
scaler=pickle.load(open("/content/drive/MyDrive/data/recent_scaler.pkl", 'rb'))

scaledData=scaler.transform(X)
scaledData=returnTextForm(scaledData)
pred_test= model.predict(scaledData)
scores2 = model.evaluate(scaledData, y, verbose=1)
print('Accuracy on the whole dataset: {:.2%} \n Error on the whole dataset: {:.2%}'.format(scores2[1], 1 - scores2[1]))


In [ ]:
Xnew=np.array([[2.04,1.81,29,36,0.38,0.42,0.484,0.428,1.08,0.68,0.72,1.01,78.3,0.73,0.39,1.07,4.4,1.03,0.66,0.68,0.99,74.3,0.68,0.56,1.03,3.9,1.01,0.71,0.7,0.95,76.9,0.65,0.5,0.91,2.6,0.99,0.67,0.66,1.01,68.0,0.67,0.36,1.01,3.4,0.91,0.7,0.69,0.81,66.6,0.56,0.5,0.80,5.2,1.04,0.67,0.69,1.04,71.1,0.69,0.26,1.03,6.4,1.04,0.68,0.67,1.07,75.1,0.68,0.56,1.00,7.4,1.04,0.72,0.69,1.05,75.9,0.71,0.64,0.99,4.4,0.88,0.71,0.65,0.78,67.9,0.58,0.52,0.82,4.3,0.86,0.72,0.65,0.86,64.5,0.56,0.48,0.78,6.6],
               [1.22,4,1,19,0.77,0.62,0.606,0.591,1.18,0.57,0.73,1.17,75.2,0.75,0.46,1.33,1.9,1.16,0.56,0.77,1.01,71.9,0.71,0.28,1.28,3.9,1.13,0.56,0.75,0.98,75.8,0.66,0.59,1.19,5.3,1.11,0.67,0.72,1.11,79.6,0.7,0.47,1.04,6.0,0.93,0.68,0.66,0.94,67.9,0.57,0.48,0.85,6.6,1.22,0.56,0.74,1.28,75.6,0.74,0.23,1.33,3.6,1.2,0.72,0.71,1.32,90.3,0.77,0.55,1.08,9.5,1.09,0.63,0.72,1.04,74.4,0.7,0.52,1.12,3.0,1.02,0.63,0.7,0.93,70.0,0.62,0.47,1.00,7.2,0.94,0.67,0.69,0.78,66.5,0.62,0.63,0.92,4.0],
               [3.18,1.33,107,51,0.58,0.48,0.5,0.472,1.26,0.62,0.75,1.34,83.0,0.81,0.27,1.30,4.9,1.1,0.72,0.7,1.18,84.6,0.73,0.37,1.00,9.8,1.06,0.72,0.7,1.13,77.3,0.73,0.4,1.02,4.5,1.02,0.67,0.69,0.98,74.6,0.67,0.46,1.00,6.4,0.89,0.63,0.69,0.69,60.9,0.55,0.56,0.87,4.0,1.12,0.61,0.71,1.1,73.9,0.72,0.28,1.18,3.6,1.07,0.7,0.68,1.1,79.0,0.71,0.48,1.02,5.3,1.07,0.7,0.66,1.13,79.9,0.72,0.54,1.03,2.8,0.98,0.68,0.68,0.93,70.2,0.65,0.53,0.95,3.7,1.06,0.67,0.68,1.06,75.75,0.7,0.46,1.05,3.85],
               [1.4,2.75,54,100,0.64,0.46,0.575,0.5,1.13,0.63,0.71,1.16,76.4,0.7,0.46,1.11,5.2,1.09,0.63,0.7,1.05,75.5,0.69,0.49,1.10,5.3,1.02,0.67,0.68,1.06,73.8,0.62,0.55,0.93,7.3,1.08,0.64,0.7,1.09,75.23,0.67,0.5,1.05,5.93,1.08,0.64,0.7,1.09,75.23,0.67,0.5,1.05,5.93,1.2,0.68,0.71,1.26,86.6,0.83,0.45,1.23,4.0,1.03,0.65,0.72,0.97,70.8,0.64,0.49,0.98,4.7,1.02,0.66,0.7,0.99,70.8,0.66,0.4,1.01,5.6,0.93,0.74,0.66,0.98,72.1,0.61,0.4,0.83,6.8,1.04,0.68,0.7,1.05,75.07,0.68,0.43,1.01,5.27],
               [1.71,2.08,121,140,0.8,0.79,0.5,0.5,1.48,0.61,0.75,1.72,99.8,0.92,0.27,1.51,3.1,1.22,0.6,0.79,1.12,78.9,0.78,0.52,1.31,4.0,1.21,0.58,0.78,1.02,82.3,0.73,0.57,1.27,5.6,1.3,0.6,0.77,1.29,87.0,0.81,0.45,1.36,4.23,1.3,0.6,0.77,1.29,87.0,0.81,0.45,1.36,4.23,1.27,0.58,0.74,1.31,86.4,0.79,0.41,1.35,5.7,1.2,0.58,0.77,1.12,77.6,0.75,0.26,1.29,6.1,1.11,0.68,0.72,1.19,79.2,0.71,0.4,1.04,9.8,1.1,0.6,0.71,1.06,74.0,0.69,0.43,1.14,8.3,1.05,0.64,0.73,0.93,74.3,0.69,0.47,1.08,6.5]])

models=list()
models.append(tf.keras.models.load_model(("/content/drive/MyDrive/data/best_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer}))
models.append(tf.keras.models.load_model(("/content/drive/MyDrive/data/test/recent_model.h5"),custom_objects={'KerasLayer':hub.KerasLayer}))
models.append(tf.keras.models.load_model(("/content/drive/MyDrive/data/test/recent_model2.h5"),custom_objects={'KerasLayer':hub.KerasLayer}))

scalers=list()
scalers.append(pickle.load(open("/content/drive/MyDrive/data/best_scaler.pkl", 'rb')))
scalers.append(pickle.load(open("/content/drive/MyDrive/data/test/recent_scaler.pkl", 'rb')))
scalers.append(pickle.load(open("/content/drive/MyDrive/data/test/recent_scaler2.pkl", 'rb')))


data=scalers[0].transform(Xnew)
data=returnTextForm(data)
predictions=models[0].predict(data)

data=scalers[1].transform(Xnew)
data=returnTextForm(data)
predictions2=models[1].predict(data)

data=scalers[2].transform(Xnew)
data=returnTextForm(data)
predictions3=models[2].predict(data)

predictionList=list()
for i in range(Xnew.shape[0]):
  yhats=list()
  yhats.append(predictions[i])
  #yhats.append(predictions2[i])
  yhats.append(predictions3[i])

  yhats = np.array(yhats)
  outcomes = np.mean(yhats)
  print(outcomes)



0.4596421
0.16127232
0.55471027
0.32457596
0.41362455
